# Sistema de Compressão

$ \frac{d\dot{m}}{dt} = \frac{A_1}{L_C}(\phi (N(t), \dot{m})P_1 - P_P(t)) $

$ \frac{d P_P}{dt} = \frac{C_1^2}{\nu _P}(\dot{m}(t) - \alpha (t) K_\nu \sqrt{P_P - P_{out}}) $

$ \begin{matrix} A_1 & = & 2.6\centerdot 10^-3 m² \\
\nu _P & = & 2.0 m³ \\
L_C & = & 2.0 m \\
K_\nu & = & \frac{0.38 kg}{(kPa)^{0.5}s} \\
P_1 & = & 4.5 MPa \\
P{out} & = & 5.0 MPa \end{matrix}
$

$ \frac{d\dot{m}}{dt} = \frac{2.6\centerdot 10^{-3}}{2.0}(1.5\centerdot 4.5 - P_P) $

$ \frac{d P_P}{dt} = \frac{479.029^2}{2.0}(\dot{m} - \alpha {0.38} \sqrt{P_P - 5.0}) $

#### Importações

In [2]:
import numpy as np
import tensorflow as tf
import sklearn as sk
from sklearn.model_selection import train_test_split
import optuna
from tensorflow import keras
from tensorflow.keras import layers
from scipy.optimize import fsolve
import casadi as ca
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
from sklearn.preprocessing import MinMaxScaler

2024-10-13 09:44:45.853733: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-13 09:44:45.856595: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-13 09:44:45.866329: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-13 09:44:45.882664: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-13 09:44:45.887493: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-13 09:44:45.898832: I tensorflow/core/platform/cpu_feature_gu

#### Constantes e Variáveis Auxiliares

In [3]:
np.random.seed(42)
print(np.random.seed)

# Constantes
A1 = (2.6)*(10**-3)
Lc = 2
kv = 0.38*100
P1 = 4.5
P_out = 5
C = 479

timestep = 2 # Passos no passado para prever o próximo
epochs = 15000
units = 50 # Número de Neurônios
nData = 600 # Segundos para cada alteração do Alfa
nAlphas = 5 # Número de vezes que o Alfa irá mudar, considere o treino e os testes.
alphas = np.random.uniform(0.35,0.65, nAlphas+1) # Abertura da válvula
perturb = 5e-4
tempo = 60
dt = 0.1 # Tempo amostral

# Variáveis auxiliares
interval = [np.linspace(i * tempo, (i + 1) * tempo, nData) for i in range(nAlphas)]
massFlowrate = []
PlenumPressure = []
alpha_values = []
RNN_train = []
RNN_trainFut = []

<built-in function seed>


### Solução Numérica

##### Cálculo da Solução

In [5]:
# Condições Iniciais
def fun(variables, A1, Lc, kv, P1, P_out, C) :
    (x,y) = variables
    eqn_1 = (A1/Lc)* ((1.5 * P1) - y)*1e6
    eqn_2 = (C**2)/2 * (x - alphas[0] * kv * np.sqrt(y*1000 - P_out*1000))
    return [eqn_1, eqn_2]

result = fsolve(fun, (0, 10), args = (A1, Lc, kv, P1, P_out, C)) 

init_m = result[0] 
init_p = result[1]
print(init_m, init_p)
# Variáveis CasADi
x = ca.MX.sym('x', 2)
alpha = ca.MX.sym('alpha', 1)

# Solução Numérica
tm1 = time.time()
for i in range(nAlphas):
    alpha_values.append(np.full(500, alphas[i]))

    rhs = ca.vertcat((A1/Lc)*((1.5 * P1) - x[1])*1e6, (C**2)/2 * (x[0] - alpha * kv * ca.sqrt(x[1]*1000 - P_out*1000)))
    ode = {'x' : x, 'ode' : rhs, 'p' : alpha }

    F = ca.integrator('F','idas', ode, interval[0][0], dt)

    for j in range(nData):
        sol = F(x0 = [init_m, init_p], p = alpha_values[i][j])
        xf_values = np.array(sol["xf"])
        aux1, aux2 = xf_values
        massFlowrate.append(aux1[0])
        PlenumPressure.append(aux2[0])
        init_m = aux1[-1]
        init_p = aux2[-1]
        RNN_train.append([aux1[0], aux2[0], alpha_values[i][j]])
        RNN_trainFut.append([aux1[0], aux2[0], alpha_values[i][j]])
    
tm2 = time.time()

massFlowrate = np.reshape(massFlowrate, [nAlphas,nData])
PlenumPressure = np.reshape(PlenumPressure, [nAlphas,nData])

print(tm2-tm1)


734.995682735719 6.75


IndexError: index 500 is out of bounds for axis 0 with size 500

##### Gráfico do Modelo

In [78]:
fig = make_subplots(rows=1, cols=3, subplot_titles=("Vazão vs Tempo", "Pressão vs Tempo", "Alpha vs Tempo"))

for i in range(0, nAlphas):
    # Vazão
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(massFlowrate[i]), mode='lines',
                             name='Vazão', legendgroup='massflow', showlegend=i == 0), row = 1, col = 1)
    # Pressão
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(PlenumPressure[i]), mode='lines',
                             name='Pressão', legendgroup='pressure', showlegend=i == 0), row = 1, col = 2)
    # Alphas
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(alpha_values[i]), mode='lines', 
                             name='Alphas', line=dict(dash='dash'), legendgroup='alpha', showlegend=i == 0), row = 1, col = 3)

# Atualiza layout
fig.update_layout(
    xaxis_title='Tempo',
    grid=dict(rows=1, columns=3),
    template='plotly',
    showlegend=False
)

# Mostra a figura
fig.show()


### Rede Neural

##### Dados de Treino

In [90]:
RNN_train = np.array(RNN_train)
RNN_trainFut = np.array(RNN_trainFut)

RNN_train = np.reshape(RNN_train, (nAlphas,3,nData))
RNN_trainFut = np.reshape(RNN_trainFut,(nAlphas,3,nData))

X_train = []
y_train = []

# Nao estou gostando de ter que separar o RNN Train e RNN Test

for j in range(nAlphas):
    for i in range(len(RNN_train[0][0])):
        X_train.append(RNN_train[j,:,i])
        if i == (len(RNN_train[0][0]) - 1):
            y_train.append(RNN_trainFut[j, :2, 0])
        else:
            y_train.append(RNN_train[j,:2,i+1])

newData = int((nData*nAlphas)/timestep)
print(newData)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_train = np.reshape(X_train, [newData,timestep,3])
y_train = np.reshape(y_train, [newData,timestep,2])


X_train_temp, X_val, y_train_temp, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Normalizar X_train e y_train
# Primeiro, precisamos "achatar" as matrizes para 2D
X_train_reshaped = X_train_temp.reshape(-1, X_train_temp.shape[-1])  # (n_samples, n_features)
y_train_reshaped = y_train_temp.reshape(-1, y_train_temp.shape[-1])  # (n_samples, n_features)

# Inicializar o MinMaxScaler
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# Ajustar e transformar os dados de entrada
X_train_normalized = scaler_X.fit_transform(X_train_reshaped).reshape(X_train_temp.shape)

# Ajustar e transformar os dados de saída
y_train_normalized = scaler_y.fit_transform(y_train_reshaped).reshape(y_train_temp.shape)

# Normalizar os dados de validação usando os mesmos parâmetros do conjunto de treino
X_val_reshaped = X_val.reshape(-1, X_val.shape[-1])
y_val_reshaped = y_val.reshape(-1, y_val.shape[-1])

X_val_normalized = scaler_X.transform(X_val_reshaped).reshape(X_val.shape)
y_val_normalized = scaler_y.transform(y_val_reshaped).reshape(y_val.shape)

print("Shape de X_train:", X_train_temp.shape)  
print("Shape de y_train:", y_train_temp.shape)  
print("Shape de X_val:", X_val.shape)      
print("Shape de y_val:", y_val.shape)

1500
Shape de X_train: (1200, 2, 3)
Shape de y_train: (1200, 2, 2)
Shape de X_val: (300, 2, 3)
Shape de y_val: (300, 2, 2)


##### Rede e Treinamento

In [ ]:

@tf.function
def lossCustom(y_true, y_pred):
    weight1 = 1
    weight2 = 1
    data_loss = tf.reduce_mean(tf.square(y_true[:, :, 0] - y_pred[:, :, 0]))* weight1 + tf.reduce_mean(tf.square(y_true[:, :, 1] - y_pred[:, :, 1]))*weight2
    return data_loss

def objective(trial):
    n_layers = trial.suggest_int("n_layers", 0, 5)  
    n_units = trial.suggest_int("n_units", 32, 512)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    # Criar o modelo
    model = keras.Sequential()
    model.add(layers.Bidirectional(layers.LSTM(n_units, input_shape=(timestep,3), return_sequences= True, activation= 'tanh')))
    model.add(layers.Dropout(0.5))
    for _ in range(n_layers):
        model.add(layers.LSTM(n_units, return_sequences=True, activation='tanh'))

    model.add(layers.Dense(2))
    model.add(layers.Lambda(lambda x: x[:, -1:, :]))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5), loss= lossCustom, metrics=['accuracy'])

    # Treinar o modelo
    model.fit(X_train_normalized, y_train_normalized, epochs=10000, batch_size=batch_size, verbose=0)

    # Avaliar o modelo
    score = model.evaluate(X_val_normalized, y_val_normalized, verbose=0)
    return score[0] 


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

print("Melhores hiperparâmetros:", study.best_params)
print("Melhor perda:", study.best_value)

In [ ]:
@tf.function
def lossCustom(y_true, y_pred):
    weight1 = 1
    weight2 = 1
    data_loss = tf.reduce_mean(tf.square(y_true[:, :, 0] - y_pred[:, :, 0]))* weight1 + tf.reduce_mean(tf.square(y_true[:, :, 1] - y_pred[:, :, 1]))*weight2
    return data_loss

optimizer = keras.optimizers.Adam(learning_rate = 0.000821443396930906)
model = keras.Sequential()
model.add(layers.Bidirectional(layers.LSTM(249, input_shape=(timestep,3), return_sequences= True, activation= 'tanh')))
model.add(layers.Dropout(0.5))
model.add(layers.LSTM(249, input_shape=(timestep,3), return_sequences= True, activation= 'tanh'))
model.add(layers.Dense(2))
model.add(layers.Lambda(lambda x: x[:, -1:, :]))
model.compile(optimizer = optimizer, loss = lossCustom)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='loss',  
    patience=units*10,     
    restore_best_weights=True 
)

history = model.fit(X_train_normalized, y_train_normalized, epochs=epochs, verbose=1, batch_size= 16 ,callbacks= [early_stopping])


##### Dados de teste

In [15]:
massFlowrateTeste = []
PlenumPressureTeste = []
RNN_test = []
x_test = []
alpha_valuesTeste = []
alphasTeste = np.random.uniform(0.35,0.65, nAlphas) # Abertura da válvula

tm1 = time.time()
for i in range(0,nAlphas):
    alpha_valuesTeste.append(alphasTeste[i] + np.random.normal(0,perturb,nData))

    rhs = ca.vertcat((A1/Lc)*((1.5 * P1) - x[1])*1e6, (C**2)/2 * (x[0] - alpha * kv * np.sqrt(x[1]*1000 - P_out*1000)))
    ode = {'x' : x, 'ode' : rhs, 'p' : alpha }
    
    F = ca.integrator('F','idas', ode, interval[0][0], dt)

    for j in range(nData):
        sol = F(x0 = [init_m, init_p], p = alpha_valuesTeste[i][j])
        xf_values = np.array(sol["xf"])
        aux1, aux2 = xf_values
        massFlowrateTeste.append(aux1)
        PlenumPressureTeste.append(aux2)
        init_m = aux1[-1]
        init_p = aux2[-1]
        RNN_test.append([aux1[0], aux2[0], alpha_valuesTeste[i][j]])

tm2 = time.time()
print(tm2-tm1)

RNN_test = np.array(RNN_test)
RNN_test = np.reshape(RNN_test, (nAlphas,3,nData))

for j in range(nAlphas):
    for i in range(len(RNN_train[0][0])):
        x_test.append(RNN_test[j,:,i])

x_test = np.array(x_test)
x_test = tf.reshape(x_test, [newData,timestep,3])
x_testN = x_test/(np.max(x_test)/2)-1

3.0036394596099854


##### Gráfico Rede Neural

In [16]:
fig = make_subplots(rows=1, cols=3, subplot_titles=("Loss","Mass Flow Rate vs Time", "Plenum Pressure vs Time"))

# Loss
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['loss']) + 1)), y=history.history['loss'], mode='lines', name='Mass Loss', line=dict(dash='solid')),
                  row=1, col=1)

tm1 = time.time()
prediction = np.array(model.predict(x_testN))
tm2 = time.time()
print(tm2-tm1)
mass = prediction[:,:,0]
pressure = prediction[:, :,1]

interval2 = [np.linspace(0, tempo*nAlphas, newData) for i in range(nAlphas)]

for i in range(nAlphas):
    for j in range(0, len(interval2[i]), nData):
        end = j + nData
        if end > len(interval2[i]):
            end = len(interval2[i])
        
        fig.add_trace(go.Scatter(x=interval2[i][j:end],y=np.squeeze(mass[j:end]),mode='lines',
                                line=dict(dash='solid')),row=1, col=2)
        fig.add_trace(go.Scatter(x=interval2[i][j:end],y=np.squeeze(pressure[j:end]),mode='lines',
                                line=dict(dash='solid')),row=1, col=3)
    # Modelo
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(massFlowrateTeste[i]), mode='lines',name='Model Mass Flow Rate', line=dict(dash='dash', color='red')),
                  row=1, col=2)
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(PlenumPressureTeste[i]), mode='lines', name= 'Model Plenum Pressure', line=dict(dash='dash', color='red')),
                  row=1, col=3)

fig.update_layout(
    title='Resultados Rede Neural',
    xaxis_title='Time',
    yaxis_title='Value',
    template='plotly',
    showlegend=False
)
fig.show()

47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 749us/step
0.07096624374389648
